In [219]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, MaxPool2D,Concatenate, Input, Conv1D, Conv2D,GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string
import csv

In [312]:
datafile = open('Resources/q.csv', 'r')
Label=[]
Ques=[]
for row in datafile:
    vals = row.split(' ',1)
    print(vals[0])
    Label.append(vals[0])
    Ques.append(vals[1])

DESC:manner
ENTY:cremat
DESC:manner
ENTY:animal
ABBR:exp
HUM:ind
HUM:gr
HUM:title
DESC:def
HUM:ind
NUM:date
DESC:reason
HUM:ind
HUM:ind
ENTY:event
LOC:state
DESC:desc
NUM:count
DESC:def
DESC:def
NUM:date
ENTY:other
HUM:ind
DESC:desc
DESC:desc
ENTY:letter
HUM:ind
LOC:other
ENTY:other
LOC:state
ABBR:exp
ENTY:other
NUM:count
HUM:gr
NUM:count
DESC:desc
NUM:date
ENTY:religion
LOC:other
ENTY:animal
DESC:manner
ENTY:other
NUM:count
NUM:count
DESC:manner
HUM:ind
ENTY:food
HUM:gr
DESC:manner
ENTY:other
LOC:country
DESC:def
HUM:ind
DESC:desc
HUM:ind
DESC:def
ENTY:color
ENTY:termeq
LOC:state
DESC:def
LOC:other
DESC:def
LOC:country
DESC:def
HUM:ind
LOC:city
DESC:def
DESC:reason
LOC:other
DESC:reason
LOC:other
HUM:ind
ENTY:other
DESC:desc
HUM:ind
DESC:manner
HUM:ind
ENTY:body
DESC:manner
LOC:country
ENTY:other
NUM:count
DESC:desc
LOC:state
ENTY:animal
NUM:count
LOC:country
ENTY:dismed
HUM:ind
NUM:date
NUM:date
DESC:reason
DESC:def
ENTY:cremat
DESC:def
DESC:def
HUM:gr
DESC:def
NUM:date
LOC:country
H

In [221]:
print(len(Label))

1606


In [222]:
print(len(Ques))

1606


In [223]:
clabel=[]
flabel=[]
txt=""
for i in range(len(Label)):
    clabel.append((Label[i].split(":"))[0])
    flabel.append((Label[i].split(":"))[1])
    

In [224]:
print(len(clabel))

1606


In [225]:
def unique(list1): 
    unique_list = [] 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

In [313]:
Unique_Labels=unique(Label)

In [316]:
dict_clabels= { Unique_clabels[i]:i for i in range(0, len(Unique_clabels) ) }

In [317]:
dict_clabels

{'DESC': 0, 'ENTY': 1, 'ABBR': 2, 'HUM': 3, 'NUM': 4, 'LOC': 5}

In [229]:
print(len(clabel))

1606


In [230]:
for n, i in enumerate(clabel):
    #print(i)
    if i=='DESC':
        clabel[n]=0
    if i=='ENTY':
        clabel[n]=1
    if i=='ABBR':
        clabel[n]=2
    if i=='HUM':
        clabel[n]=3
    if i=='NUM':
        clabel[n]=4  
    if i=='LOC':
        clabel[n]=5
    

In [231]:
print(len(clabel))

1606


In [232]:
MAX_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
# the percentage of train test split to be applied
VALIDATION_SPLIT = 0.20
# the dimension of vectors to be used
EMBEDDING_DIM = 100
# filter sizes of the different conv layers 
filter_sizes = [3,4,5]
num_filters = 512
embedding_dim = 100
# dropout probability
drop = 0.5
batch_size = 30
epochs =30

In [233]:
from keras.utils import to_categorical
tokenizer  = Tokenizer(num_words = MAX_WORDS)
print(tokenizer.fit_on_texts(Ques))
sequences =  tokenizer.texts_to_sequences(Ques)
print(len(sequences))
word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(clabel))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
print(len(labels))

None
1606
unique words : 3874
Shape of data tensor: (1606, 1000)
Shape of label tensor: (1606, 6)
1606


In [234]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [235]:
embeddings_index = {}
GLOVE_DIR='./GLOVE.6B/'
FILE=os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')
f = open(FILE,encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()


print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [236]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [237]:
print(embedding_matrix.shape)

(3875, 100)


In [238]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                               input_length=MAX_SEQUENCE_LENGTH,
                             trainable=False)

In [241]:
inputs = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding = embedding_layer(inputs)

print(embedding.shape)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(embedding)
print(reshape.shape)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal',activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal',activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal',activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] +
                                 1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)


concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=6, activation='softmax')(dropout)
print(output)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

(None, 1000, 100)
(None, 1000, 100, 1)
Tensor("dense_3/Softmax:0", shape=(None, 6), dtype=float32)
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1000, 100)    387500      input_6[0][0]                    
__________________________________________________________________________________________________
reshape_6 (Reshape)             (None, 1000, 100, 1) 0           embedding_4[1][0]                
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 998, 1, 512)  154112      reshape_6[0][0]            

In [245]:
print("Traning Model...")
model.fit(x_train, y_train, batch_size=10, epochs=50, verbose=1, callbacks=[checkpoint])
score = model.evaluate(x_test, y_test, verbose=0)  
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])



Traning Model...
Epoch 1/50
1285/1285 [==============================] - 115s 89ms/step - loss: 0.1149 - accuracy: 0.9837
Epoch 2/50
1285/1285 [==============================] - 118s 92ms/step - loss: 0.0981 - accuracy: 0.9860
Epoch 3/50
1285/1285 [==============================] - 118s 92ms/step - loss: 0.0859 - accuracy: 0.9922
Epoch 4/50
1285/1285 [==============================] - 117s 91ms/step - loss: 0.0739 - accuracy: 0.9938
Epoch 5/50
1285/1285 [==============================] - 117s 91ms/step - loss: 0.0617 - accuracy: 0.9977
Epoch 6/50
1285/1285 [==============================] - 117s 91ms/step - loss: 0.0574 - accuracy: 0.9953
Epoch 7/50
1285/1285 [==============================] - 119s 92ms/step - loss: 0.0512 - accuracy: 0.9984
Epoch 8/50
1285/1285 [==============================] - 118s 92ms/step - loss: 0.0541 - accuracy: 0.9977
Epoch 9/50
1285/1285 [==============================] - 118s 91ms/step - loss: 0.0452 - accuracy: 0.9992
Epoch 10/50
1285/1285 [===============

In [247]:
predictions = model.predict(x_test, batch_size=1024,   verbose=1)
labels = ['DESC', 'ENTY', 'ABBR', 'HUM', 'NUM', 'LOC']
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

321/321 [==============================] - 14s 43ms/step


In [248]:
prediction_labels

['LOC',
 'ENTY',
 'ENTY',
 'HUM',
 'ENTY',
 'ENTY',
 'LOC',
 'HUM',
 'ENTY',
 'ENTY',
 'HUM',
 'DESC',
 'DESC',
 'HUM',
 'DESC',
 'HUM',
 'NUM',
 'LOC',
 'ENTY',
 'LOC',
 'HUM',
 'ENTY',
 'ENTY',
 'ENTY',
 'DESC',
 'LOC',
 'HUM',
 'ENTY',
 'NUM',
 'ENTY',
 'DESC',
 'HUM',
 'DESC',
 'DESC',
 'NUM',
 'NUM',
 'NUM',
 'HUM',
 'LOC',
 'HUM',
 'LOC',
 'HUM',
 'HUM',
 'LOC',
 'ENTY',
 'ENTY',
 'DESC',
 'ENTY',
 'NUM',
 'ENTY',
 'LOC',
 'DESC',
 'HUM',
 'ENTY',
 'NUM',
 'HUM',
 'ENTY',
 'ENTY',
 'NUM',
 'DESC',
 'ENTY',
 'ENTY',
 'ENTY',
 'LOC',
 'HUM',
 'ENTY',
 'ENTY',
 'NUM',
 'NUM',
 'ENTY',
 'HUM',
 'NUM',
 'NUM',
 'LOC',
 'HUM',
 'HUM',
 'ENTY',
 'HUM',
 'HUM',
 'LOC',
 'LOC',
 'ENTY',
 'LOC',
 'LOC',
 'ENTY',
 'ENTY',
 'ABBR',
 'NUM',
 'HUM',
 'HUM',
 'LOC',
 'HUM',
 'HUM',
 'LOC',
 'ENTY',
 'ENTY',
 'ENTY',
 'ENTY',
 'NUM',
 'NUM',
 'DESC',
 'ENTY',
 'ENTY',
 'HUM',
 'LOC',
 'HUM',
 'DESC',
 'ENTY',
 'ENTY',
 'HUM',
 'ENTY',
 'ENTY',
 'ENTY',
 'DESC',
 'HUM',
 'ENTY',
 'ENTY',
 'DESC',

In [283]:
q=['What films featured the character Popeye Doyle ?','What contemptible scoundrel stole the cork from my lunch ?','What is the full form of .com ?']

In [307]:
from keras.utils import to_categorical
tokenizer  = Tokenizer(num_words = MAX_WORDS)
print(tokenizer.fit_on_texts(q))
sequences =  tokenizer.texts_to_sequences(q)
print(len(sequences))
word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

None
3
unique words : 19


In [308]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]

In [309]:
predictions = model.predict(data, batch_size=1024,   verbose=1)

3/3 [==============================] - 0s 43ms/step


In [310]:
predicted_label=[]
for p in predictions:
    predicted_label.append(labels[np.argmax(p)])

In [311]:
predicted_label

['ENTY', 'ENTY', 'HUM']